In [90]:
import tensorflow
import keras
from keras import layers as L
import ssl

In [92]:
def build_model(input_shape, num_classes=3):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Backbone """
    backbone = keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        alpha=1.0
    )
    backbone.trainable = True
    # backbone.summary()

    """ Detection Head """
    x = backbone.output
    x = L.Conv2D(256, kernel_size=1, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.5)(x)
    bbox = L.Dense(4, activation="sigmoid", name="bbox")(x)
    label = L.Dense(num_classes, activation="softmax", name="label")(x)

    """ Model """
    model = keras.models.Model(inputs=[inputs], outputs=[bbox, label])
    return model

In [93]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from sklearn.model_selection import train_test_split

In [94]:
def createTrainAndTest():
    thing = pd.read_csv("../../images/version1/Labeled/Full/Version_4_27_24/Full_BB_Images.csv")
    train, test = train_test_split(thing, train_size=0.8, test_size=0.2)
    train.to_csv("../../images/version1/Labeled/Full/Version_4_27_24/Train_BB_Images.csv")
    test.to_csv("../../images/version1/Labeled/Full/Version_4_27_24/Test_BB_Images.csv")

In [95]:
# Global Parameters
height = 218
width = 384
num_classes = 3

In [96]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [97]:
def load_labels(path):
    return ["H", "S", "U"]

In [98]:
def read_image_bbox(path, bbox, label_index):
    path = path.decode()
    image = cv2.imread(path)
    h, w, _ = image.shape
    image = cv2.resize(image, (width, height))
    image = (image - 127.5) / 127.5
    image = image.astype(np.float32)

    x1, y1, x2, y2 = bbox
    norm_x1 = float(x1/w)
    norm_y1 = float(y1/h)
    norm_x2 = float(x2/w)
    norm_y2 = float(y2/h)
    norm_bbox = np.array([norm_x1, norm_y1, norm_x2, norm_y2], dtype=np.float32)

    label = [0] * num_classes
    label[label_index] = 1
    class_label = np.array(label, dtype=np.float32)

    return image, norm_bbox, class_label

In [99]:
def parse(image, bbox, label):
    image, bbox, label = tf.numpy_function(read_image_bbox, [image, bbox, label], [tf.float32, tf.float32, tf.float32])
    image.set_shape((height, width, 3))
    bbox.set_shape((4))
    label.set_shape((num_classes))

    return (image), (bbox, label)

In [100]:
def tf_dataset(images, bboxes, labels, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((images, bboxes, labels))
    ds = ds.map(parse).batch(batch).prefetch(10)
    return ds


In [101]:
def load_data(path, classes, train=True):
    images = []
    bboxes = []
    labels = []

    if train == True:
        df = pd.read_csv("../../images/version1/Labeled/Full/Version_4_27_24/Train_BB_Images.csv")
    else:
        df = pd.read_csv("../../images/version1/Labeled/Full/Version_4_27_24/Test_BB_Images.csv")
    
    for index, row in df.iterrows():
        if index == 0:
            continue
        name = row.iloc[2]
        width = int(row.iloc[3])
        height = int(row.iloc[4])
        x1 = int(row.iloc[5])
        y1 = int(row.iloc[6])
        x2 = int(row.iloc[7])
        y2 = int(row.iloc[8])
        label = int(row.iloc[9])
        label_name = classes[label]

        name = name + ".jpg"
        image = os.path.join(path, "images", name)
        bbox = [x1, y1, x2, y2]

        images.append(image)
        bboxes.append(bbox)
        labels.append(label)
    return images, bboxes, labels

In [102]:
def load_dataset(path, classes, split=0.2):
    ## train, validation, testing
    train_images, train_bboxes, train_labels = load_data(path, classes, train=True)

    split_size = int(len(train_images) * split)
    train_images, valid_images = train_test_split(train_images, test_size=split_size, random_state=42)
    train_bboxes, valid_bboxes = train_test_split(train_bboxes, test_size=split_size, random_state=42)
    train_labels, valid_labels = train_test_split(train_labels, test_size=split_size, random_state=42)

    test_images, test_bboxes, test_labels = load_data(path, classes, train=False)

    return (train_images, train_bboxes, train_labels), (valid_images, valid_bboxes, valid_labels), (test_images, test_bboxes, test_labels)

In [109]:
def main():
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir("files")

    height = 218
    width = 384
    batch_size = 16
    lr = 0.001
    num_epochs = 200

    model_path = "model.weights.h5"
    csv_path = "log.csv"
    dataset_path = "../../images/version1/Labeled/Full/Version_4_27_24/"

    classes = load_labels(dataset_path)
    num_classes = len(classes)

    (train_images, train_bboxes, train_labels), (valid_images, valid_bboxes, valid_labels), (test_images, test_bboxes, test_labels) = load_dataset(dataset_path, classes, split=0.2)
    print(f"{len(train_images)} - {len(train_bboxes)} - {len(train_labels)}")
    print(f"{len(valid_images)} - {len(valid_bboxes)} - {len(valid_labels)}")
    print(f"{len(test_images)} - {len(test_bboxes)} - {len(test_labels)}")

    train_ds = tf_dataset(train_images, train_bboxes, train_labels, batch=batch_size)
    valid_ds = tf_dataset(valid_images, valid_bboxes, valid_labels, batch=batch_size)

    model = build_model((height, width, 3))
    model.load_weights(model_path)
    model.compile(
        loss = {
            "bbox": "binary_crossentropy",
            "label": "categorical_crossentropy"
        },
        optimizer=keras.optimizers.Adam(lr)
    )

    callbacks1 = [
        keras.callbacks.ModelCheckpoint(model_path, verbose=1, save_weights_only=True, save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        keras.callbacks.CSVLogger(csv_path, append=True),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
    ]

    model.fit(
        train_ds,
        epochs=num_epochs,
        validation_data=valid_ds,
        callbacks=callbacks1
    )


In [110]:
main()

329 - 329 - 329
82 - 82 - 82
103 - 103 - 103


/var/folders/4w/7mmrnj315pzgcqvpx7n96sf40000gn/T/ipykernel_41426/2947174705.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = keras.applications.MobileNetV2(


Epoch 1/200
 3/21 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - loss: 0.6236

KeyboardInterrupt: 